# Install Pachages

In [ ]:
!pip install pygame

In [ ]:
!pip install opencv-contrib-python

In [ ]:
!pip install gTTS

# Initializing database

In [ ]:
import os
import sys
import cv2
import numpy as np
import pickle


def normalize(X, low, high, dtype=None):
    """Normalizes a given array in X to a value between low and high."""
    X = np.asarray(X)
    minX, maxX = np.min(X), np.max(X)
    # normalize to [0...1].
    X = X - float(minX)
    X = X / float((maxX - minX))
    # scale to [low...high].
    X = X * (high - low)
    X = X + low
    if dtype is None:
        return np.asarray(X)
    return np.asarray(X, dtype=dtype)


def read_images(path, sz=None):
    """Reads the images in a given folder, resizes images on the fly if size is given.

    Args:
        path: Path to a folder with subfolders representing the subjects (persons).
        sz: A tuple with the size Resizes

    Returns:
        A list [X,y]

            X: The images, which is a Python list of numpy arrays.
            y: The corresponding labels (the unique number of the subject, person) in a Python list.
    """
    c = 0
    X, y, folder = [], [], []
    for dirname, dirnames, filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                try:
                    im = cv2.imread(os.path.join(subject_path, filename), cv2.IMREAD_GRAYSCALE)
                    # resize to given size (if given)
                    if (sz is not None):
                        im = cv2.resize(im, sz)
                    X.append(np.asarray(im, dtype=np.uint8))
                    y.append(c)
                    print(os.path.split(subject_path)[1])
                    folder.append(os.path.split(subject_path)[1])
                except IOError as err:
                    print ("I/O error: {0}".format(err))
                except:
                    print ("Unexpected error:", sys.exc_info()[0])
                    raise
            c = c + 1
    return [X, y, folder]

# Load Images

In [ ]:
[X, y, names] = read_images(r'.\att_faces')
y = np.asarray(y, dtype=np.int32)

# Training Model

In [ ]:
model = cv2.face.EigenFaceRecognizer_create()
model.train(np.asarray(X), np.asarray(y))

# Save Model

In [ ]:
model.save("faces.yml")

# Save Data

In [ ]:
set(names)

In [ ]:
for i, n in enumerate( names):
    if n == 'somebody':
        names[i] = 'John'

In [ ]:
set(names)

In [ ]:
with open("names.txt", 'wb') as f:
    pickle.dump(names, f)
with open("y.txt", 'wb') as f:
    pickle.dump(y, f)

# Recognizing faces

In [ ]:
import cv2
import pickle
from TextToSpeech import TextToSpeech


def getName(model, y, names, img):
    [p_label, p_confidence] = model.predict(img)
    return (names[y.tolist().index(p_label)], p_confidence)

model = cv2.face.EigenFaceRecognizer_create()

model.read("faces.yml")
with open("names.txt", 'rb') as f:
    names = pickle.load(f)
with open("y.txt", 'rb') as f:
    indices = pickle.load(f)

cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

video_capture = cv2.VideoCapture(0)
inx = 0

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        head = gray[y:y + h, x:x + w]
        sX = 92.0 / w
        sY = 112.0 / w
        head = cv2.resize(head, None, fx=sX, fy=sY, interpolation=cv2.INTER_CUBIC)
        cv2.imshow('Clip', head)
        cv2.moveWindow('Clip', 700, 0)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        name, conf = getName(model, indices, names, head)
        if conf < 5000:
            cv2.putText(frame, str(names.index(name)), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0))
        else:
            cv2.putText(frame, "???", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255))
    
    # Display the resulting frame
    cv2.imshow('Video', frame)
    cv2.moveWindow('Video', 0, 0)
    
    k = cv2.waitKey(1) & 0xFF
    if k == ord('q'):
        break
    elif k == ord('s'):
        fileName = './FaceCaptured/' + str(inx) + '.pgm'
        print( fileName)
        cv2.imwrite(fileName, head)
        inx += 1
    elif k == ord('i'):
        name, conf = getName(model, indices, names, head)
        print( " %s %.2f " % (name, conf))
        strToSay = u"你好嗎" + name
        TextToSpeech.saySomthing(strToSay, "zh-tw")
        
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()